In [1]:
import pandas as pd
from torch.utils.data import Dataset, random_split
import torch

d:\python\python3.10.8\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
info_collection_csv_data_file = "信息搜集 04d99d2630ca48a3b020727db11c6381.csv"
info_collection_pd = pd.read_csv(info_collection_csv_data_file)

In [3]:
print(info_collection_pd.columns)
# info_collection_pd["Tags"]
for i in zip(info_collection_pd["Summary"],info_collection_pd["Tags"]):
    print(i)
    break

Index(['Title', 'Tags', 'URL', 'Found_date', 'Summary'], dtype='object')
('Google开源漏洞扫描', 'google, scan, vulnerable')


In [4]:
max_length = 200

In [34]:
class Info_collection_dataset(Dataset):
    def __init__(self, ic_pd, tokenizer, max_length) -> None:
        super().__init__()
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for i in zip(ic_pd["Summary"],ic_pd["Tags"]):
            if i[0] == "" or i[1] == "": continue
            if i[0] == "nan" or i[1] == "nan":continue
            encodings_dict = tokenizer('<|startoftext|>' + f"Summary:{i[0]}\nTags:{i[1]}" + '<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]  

class Info_collection_dataset_bloom(Dataset):
    def __init__(self, ic_pd, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for i in zip(ic_pd["Summary"],ic_pd["Tags"]):
            if i[0] == "" or i[1] == "": continue
            if i[0] == "nan" or i[1] == "nan":continue
            encodings_dict = tokenizer(f"Summary:{i[0]}" , truncation=True,
                                       max_length=max_length, padding="max_length")
            
            print(i)
            print(len(encodings_dict['input_ids']))

            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
            # self.labels.append(f"Tags:{i[1]}")
            
            self.labels.append(tokenizer.encode(f"Tags:{i[1]}", return_tensors="pt"))

    def __getitem__(self, index):
        # text = self.data[index]["text"]
        # label = self.data[index]["label"]
        # encoding = self.tokenizer(text, truncation=True, padding=True)
        # return {
        #     "input_ids": self.input_ids[index],
        #     "attention_mask": self.attn_masks[index],
        #     "labels":
        # }
        return self.input_ids[index], self.attn_masks[index], self.labels[index]

    def __len__(self):
        return len(self.input_ids)         

In [39]:
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM, IntervalStrategy

torch.manual_seed(42)
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B", bos_token='<|startoftext|>',
#                                           eos_token='<|endoftext|>', pad_token='<|pad|>')
# model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B").cuda()
# model.resize_token_embeddings(len(tokenizer))

# model_name = "bigscience/bloom-560m"
model_name = "EleutherAI/gpt-neo-125M"
# model_name = "google/mt5-small"
# model_name = "gpt2"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')

# model = AutoModelForCausalLM.from_pretrained(model_name).half()
model = AutoModelForCausalLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"
model.to(device)

input_text = "Summary:通过特殊关键词，在google上搜索用于渗透测试以及安全研究的公开信息。\nTags:"

# Encode the input text
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate text
output = model.generate(input_ids.to(device),
    max_length = 150,
    temperature = 1.0,
    repetition_penalty = 3.0,
    do_sample = True
)

# Decode the output text
output_text = tokenizer.decode(output[0], skip_special_tokens=False)

print(output_text)


model.resize_token_embeddings(len(tokenizer))
model.train()


loading configuration file config.json from cache at D:\python\huggingface_cache\models--EleutherAI--gpt-neo-125M\snapshots\b983397156c0991016feccfbcbe1fe2746d47b29\config.json
Model config GPTNeoConfig {
  "_name_or_path": "EleutherAI/gpt-neo-125M",
  "activation_function": "gelu_new",
  "architectures": [
    "GPTNeoForCausalLM"
  ],
  "attention_dropout": 0,
  "attention_layers": [
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local"
  ],
  "attention_types": [
    [
      [
        "global",
        "local"
      ],
      6
    ]
  ],
  "bos_token_id": 50256,
  "embed_dropout": 0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": null,
  "layer_norm_epsilon": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neo",
  "num_heads": 12,
  "num_layers": 12,
  "resid_dropout": 0,
  "sum

Summary:通过特殊关键词，在google上搜索用于渗透测试以及安全研究的公开信息。
Tags: 解决适应代理大策略规律

https://blog.gravatarapi.com/1/img/63fafef2b7ed78b5e6d8c0ae93764a3df_s.html

https://github.com/go-


GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_features=3

In [38]:
dataset = Info_collection_dataset_bloom(info_collection_pd, tokenizer,max_length)

train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
training_args = TrainingArguments(output_dir='./results', num_train_epochs=5, logging_steps=5000,
                                  save_strategy=IntervalStrategy.NO,
                                  per_device_train_batch_size=2, per_device_eval_batch_size=2,
                                  warmup_steps=100, weight_decay=0.01, logging_dir='./logs')
Trainer(model=model, args=training_args, train_dataset=train_dataset,
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()


Using pad_token, but it is not set yet.
  0%|          | 0/280 [16:26<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\lordsimon\AppData\Local\Temp\ipykernel_5988\335980306.py:1 in <module>                  │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\lordsimon\\AppData\\Local\\Temp\\ipykernel_5988\\335980306.py'                       │
│                                                                                                  │
│ C:\Users\lordsimon\AppData\Local\Temp\ipykernel_5988\227901482.py:28 in __init__                 │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\lordsimon\\AppData\\Local\\Temp\\ipykernel_5988\\227901482.py'                       │
│                                                                                                  │
│ d:\python\python3.10.8\lib\site-packages\transformers\tokenization_utils_base.py:2523 in         │
│ __call__                                                                                         │
│                                                                                                  │
│   2520 │   │   │   # input mode in this case.                                                    │
│   2521 │   │   │   if not self._in_target_context_manager:                                       │
│   2522 │   │   │   │   self._switch_to_input_mode()                                              │
│ ❱ 2523 │   │   │   encodings = self._call_one(text=text, text_pair=text_pair, **all_kwargs)      │
│   2524 │   │   if text_target is not None:                                                       │
│   2525 │   │   │   self._switch_to_target_mode()                                                 │
│   2526 │   │   │   target_encodings = self._call_one(text=text_target, text_pair=text_pair_targ  │
│                                                                                                  │
│ d:\python\python3.10.8\lib\site-packages\transformers\tokenization_utils_base.py:2629 in         │
│ _call_one                                                                                        │
│                                                                                                  │
│   2626 │   │   │   │   **kwargs,                                                                 │
│   2627 │   │   │   )                                                                             │
│   2628 │   │   else:                                                                             │
│ ❱ 2629 │   │   │   return self.encode_plus(                                                      │
│   2630 │   │   │   │   text=text,                                                                │
│   2631 │   │   │   │   text_pair=text_pair,                                                      │
│   2632 │   │   │   │   add_special_tokens=add_special_tokens,                                    │
│                                                                                                  │
│ d:\python\python3.10.8\lib\site-packages\transformers\tokenization_utils_base.py:2693 in         │
│ encode_plus                                                                                      │
│                                                                                                  │
│   2690 │   │   """                                                                               │
│   2691 │   │                                                                                     │
│   2692 │   │   # Backward compatibility for 'truncation_strategy', 'pad_to_max_length'           │
│ ❱ 2693 │   │   padding_strategy, truncation_strategy, max_l

In [8]:
model.eval()

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250680, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0): BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
      (1): BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementw

In [9]:
# input_text = "请补全‘？’部分：白日依山尽，？？？？？；？？？？？，更上一层楼"
# input_text = "生成一段python脚本。用来生成一个0到42之间的随机数。"
input_text = "Summary:通过特殊关键词，在google上搜索用于渗透测试以及安全研究的公开信息。\nTags:"

# Encode the input text
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate text
output = model.generate(input_ids.to(device),
    do_sample=True, top_k=50,
                                bos_token='<|startoftext|>',
                                eos_token='<|endoftext|>', pad_token='<|pad|>',
                                max_length=300, top_p=0.95, temperature=1.9, num_return_sequences=20
)

# Decode the output text
output_text = tokenizer.decode(output[0], skip_special_tokens=False)

print(output_text)

Summary:通过特殊关键词，在google上搜索用于渗透测试以及安全研究的公开信息。
Tags: securityresearch., top research services, google analytics. The first and the final term is given; both have a suffix -ing but without an ending in -dented
I had to get together with my mother for Sunday morning because our son was having some kind of epileptic seizures.
This happened when I picked him up from school at 6:20am or so. My hubby (also pediatrician) noticed that about 10 mins later he got scared all weirdly, then his jaw looked like it couldn't stop falling off.
My mom called back yesterday telling me something she would tell Sometone who knew her daughter's father. (


In [10]:
# generated = tokenizer("<s>", return_tensors="pt").input_ids.cuda()
# sample_outputs = model.generate(generated, do_sample=True, top_k=50,

#                                 max_length=300, top_p=0.95, temperature=1.9, num_return_sequences=20)
# for i, sample_output in enumerate(sample_outputs):
#     print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))